In [1]:
import os


In [5]:
%pwd

'c:\\Users\\kolap\\Documents\\Mini_Project\\Chicken-Disease-Detection-Using-MLOPS'

In [4]:
os.chdir("../")

In [48]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [49]:
import sys
import os

# Add the 'src' directory to the PYTHONPATH
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), 'src')))

# Now you should be able to import from the chickendisease module
from chickendisease.constants import *
from chickendisease.utils.common import read_yaml, create_directories


In [50]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config


In [51]:
import os
import urllib.request as request
import zipfile
from chickendisease import logger
from chickendisease.utils.common import get_size

In [52]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  


    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
from pathlib import Path

config_path = Path("config/config.yaml")
if config_path.exists():
    print(f"YAML file exists at: {config_path.resolve()}")
else:
    raise FileNotFoundError(f"YAML file not found at {config_path}")



In [54]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-18 18:32:12,970: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-18 18:32:12,977: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-18 18:32:12,978: INFO: common: created directory at: artifacts]
[2024-10-18 18:32:12,979: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-18 18:32:18,542: INFO: 4037744573: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 11636724
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "fc25038fdcaeff0382b2e623b7c4340b9fc213e6d16f7101897bb5d0cb2b9557"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0D7A:2E3769:B822C:E3E10:67125C49
Accept-Ranges: bytes
Date: Fri, 18 Oct 2024 13:02:03 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4722-BOM
X-Cache: MIS